In [1]:
!pip install pandas pyodbc sqlalchemy


In [2]:
import pandas as pd
from sqlalchemy import create_engine

In [3]:
# Replace 'your_file.csv' with your CSV file path
df = pd.read_csv('olist_order_payments_dataset.csv')

# Optional: check the first few rows
df.head()


,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78
4,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45


In [17]:
drivers = [d for d in pyodbc.drivers() if "SQL Server" in d]
if not drivers:
    raise Exception("No SQL Server ODBC driver found! Install ODBC Driver 17 or 18 for SQL Server.")
driver = drivers[-1]
print(f"Using ODBC driver: {driver}")

Using ODBC driver: ODBC Driver 18 for SQL Server


In [18]:
import pyodbc

server = 'ecommerce-db-server.database.windows.net'
database = 'OlistProject'
username = 'sqladmin'
password = 'Team@123'
driver = 'ODBC Driver 18 for SQL Server'

conn_str = (
    f"Driver={{{driver}}};"
    f"Server={server},1433;"  # Azure SQL requires port 1433
    f"Database={database};"
    f"Uid={username};"
    f"Pwd={password};"
    "Encrypt=yes;"
    "TrustServerCertificate=no;"
    "Connection Timeout=30;"
)

conn = pyodbc.connect(conn_str)
cursor = conn.cursor()
print("Connected to Azure SQL successfully!")


Connected to Azure SQL successfully!


In [19]:
cursor.execute("""
IF OBJECT_ID('dbo.olist_order_payments_dataset', 'U') IS NULL
CREATE TABLE dbo.olist_order_payments_dataset (
    order_id VARCHAR(50),
    payment_sequential INT,
    payment_type VARCHAR(50),
    payment_installments INT,
    payment_value DECIMAL(10,2)
)
""")
conn.commit()

In [20]:
batch_size = 500  # adjust as needed
data_tuples = list(df.itertuples(index=False, name=None))

for i in range(0, len(data_tuples), batch_size):
    batch = data_tuples[i:i+batch_size]
    cursor.executemany("""
        INSERT INTO dbo.olist_order_payments_dataset
        (order_id, payment_sequential, payment_type, payment_installments, payment_value)
        VALUES (?, ?, ?, ?, ?)
    """, batch)
    conn.commit()
    print(f"Inserted rows {i} to {i + len(batch) - 1}")

cursor.close()
conn.close()
print("CSV uploaded to Azure SQL successfully in batches!")

Inserted rows 0 to 499
Inserted rows 500 to 999
Inserted rows 1000 to 1499
Inserted rows 1500 to 1999
Inserted rows 2000 to 2499
Inserted rows 2500 to 2999
Inserted rows 3000 to 3499
Inserted rows 3500 to 3999
Inserted rows 4000 to 4499
Inserted rows 4500 to 4999
Inserted rows 5000 to 5499
Inserted rows 5500 to 5999
Inserted rows 6000 to 6499
Inserted rows 6500 to 6999
Inserted rows 7000 to 7499
Inserted rows 7500 to 7999
Inserted rows 8000 to 8499
Inserted rows 8500 to 8999
Inserted rows 9000 to 9499
Inserted rows 9500 to 9999
Inserted rows 10000 to 10499
Inserted rows 10500 to 10999
Inserted rows 11000 to 11499
Inserted rows 11500 to 11999
Inserted rows 12000 to 12499
Inserted rows 12500 to 12999
Inserted rows 13000 to 13499
Inserted rows 13500 to 13999
Inserted rows 14000 to 14499
Inserted rows 14500 to 14999
Inserted rows 15000 to 15499
Inserted rows 15500 to 15999
Inserted rows 16000 to 16499
Inserted rows 16500 to 16999
Inserted rows 17000 to 17499
Inserted rows 17500 to 17999
In